# Selection et entrainement des modèles 

***Résumé Exécutif du Feature Engineering - Prévision Solaire J+1***

L'étape de feature_engineering a consisté à la prise en compte des conclusions de l'analyse exploratoire (EDA), ainsi que la création de garde-fous données manquantes/outliers ainsi que la création de features cycliques et de variables laggées.

**Données** : Dataset horaire sur 2.5 ans (01/2023 - 05/2025) de production RTE et prévisions OpenMétéo. Identification de 2 valeurs manquantes nocturnes dans la cible, qui seront imputées à 0.

**Gardes-fous :**
- Création de tests IQR/Z-score et raise des outliers en intersection des deux filtres ;
- Interpolation des séquences temporelles inférieures à 3h consécutives ;
- Lors d'une absence de séquence de plus de 3 heures, création d'un reporting des séquences les plus longues, et potentiellement création d'un futur algorithme KNN - Filtre de Kalman.

**Features créées :**
- Création de features cycliques heures + mois, en fonction de la saisonnalité du cycle solaire ;
- Création de features laggées (data leakage évité): 
  - Retard de 24, 32 et 48 (observation des cross-correlation + cohérent physiquement) ;
  - Moyennes mobiles de 24, 32 et 48 périodes (analogue aux features retard).
 
**Etapes effectuées dans ce notebook** :

- Transformations statistiques pour les données LSTM et/ou SARIMAX si nécessaire ;
- Baseline SARIMAX avec les features physiques les plus corrélées ;
- Sélection de features (Embedding via LightGBM) ;
- Validation croisée "Expanding Window" avec optimisation des hyperparamètres (Optuna) ;
- Développement de modèles LightGBM et LSTM, avec MC dropout et regression quantile pour quantifier l'incertitude des modèles.

### Import des librairies

In [1]:
# Base
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from typing import Optional, Dict
import logging

# Modèles
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from scipy.stats import jarque_bera, shapiro
import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error

In [2]:
# Confirmation d'être à la racine du dossier
project_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
os.chdir(project_root)

#### Préparation des datasets d'entrainement

In [3]:
df = pd.read_csv("data/processed/df_engineered.csv", index_col=0)
df.index = pd.to_datetime(df.index, utc=True).tz_convert("Europe/Paris")

In [4]:
def prepare_data_for_nn(df: pd.DataFrame, target_col: str, features_to_use: Optional[list[str]] = None):
    """Retourne pour dataframe df les données d'entrée du modèle X transformée par MinMaxScaler (X_scaled) et la colonne cible (target_col)

    Args:
        df (pd.DataFrame): DataFrame df avec toutes les colonnes, colonne cible comprise.
        target_col (str): Colonne cible.
        features_to_use (Optional[list[str]], optional): Features à utiliser (optionnel). Defaults to None.

    Returns:
        X_scaled (pd.DataFrame), y (pd.Series) : DataFrame scalé et colonne cible.
    """
    if features_to_use:
        X = df[features_to_use]
    else:
        X = df.drop(columns=target_col)
    y = df[target_col]

    # Scaling des features
    scaler = MinMaxScaler()
    X_scaled = pd.DataFrame(scaler.fit_transform(X), index = X.index, columns=X.columns)

    return X_scaled, y

In [5]:
def time_series_fold(X: pd.DataFrame, y: pd.Series):
    
    """_summary_

    Args:
        X (pd.DataFrame): _description_
        y (pd.Series): _description_

    Returns:
        _type_: _description_
    """

    dict_ts_fold = {}
    tscv = TimeSeriesSplit(gap=0, n_splits=5)
    X_array = np.array(X) 
    y_array = np.array(y)

    for i, (train_index, test_index) in enumerate(tscv.split(X)):
        X_train, X_test = X_array[train_index], X_array[test_index]
        y_train, y_test = y_array[train_index], y_array[test_index] 
        dict_ts_fold[f'train_{i}'] = [X_train, y_train]
        dict_ts_fold[f'test_{i}'] = [X_test, y_test]
    
    return dict_ts_fold

In [6]:
df.columns

Index(['solar_mw', 'temperature_2m', 'relative_humidity_2m', 'precipitation',
       'surface_pressure', 'cloud_cover', 'wind_speed_10m',
       'wind_direction_10m', 'global_tilted_irradiance',
       'temperature_2m_delta_minmax', 'temperature_2m_std',
       'relative_humidity_2m_delta_minmax', 'relative_humidity_2m_std',
       'precipitation_delta_minmax', 'precipitation_std',
       'surface_pressure_delta_minmax', 'surface_pressure_std',
       'cloud_cover_delta_minmax', 'cloud_cover_std',
       'wind_speed_10m_delta_minmax', 'wind_speed_10m_std',
       'wind_direction_10m_delta_minmax', 'wind_direction_10m_std',
       'global_tilted_irradiance_delta_minmax', 'global_tilted_irradiance_std',
       'month_sin', 'month_cos', 'hour_sin', 'hour_cos',
       'global_tilted_irradiance_lag_24', 'global_tilted_irradiance_ma_24',
       'global_tilted_irradiance_lag_32', 'global_tilted_irradiance_ma_32',
       'global_tilted_irradiance_lag_48', 'global_tilted_irradiance_ma_48',
    

In [7]:
feature_sarimax = ['temperature_2m', 'relative_humidity_2m', 'precipitation', 
        'cloud_cover', 'wind_speed_10m',
       'wind_direction_10m', 'global_tilted_irradiance',
       'global_tilted_irradiance_delta_minmax', 'global_tilted_irradiance_std']

In [8]:
X = df[feature_sarimax]
y = df["solar_mw"]
scaler = StandardScaler()
folds = time_series_fold(X, y)